<a href="https://colab.research.google.com/github/shahadMAlshalawi/BiT-ImageCaptioning/blob/main/notebooks/features_extraction_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Features Extraction Demo



---



In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## BiT Install

In [ ]:
%cd /content
!git clone https://github.com/shahadMAlshalawi/BiT-ImageCaptioning.git
%cd /content/BiT-ImageCaptioning
!pip install -e . --quiet
import IPython
app = IPython.Application.instance()
_=app.kernel.do_shutdown(True)

/content
Cloning into 'BiT-ImageCaptioning'...
remote: Enumerating objects: 738, done.
remote: Counting objects: 100% (112/112), done.
remote: Compressing objects: 100% (89/89), done.
remote: Total 738 (delta 61), reused 60 (delta 23), pack-reused 626 (from 1)
Receiving objects: 100% (738/738), 433.05 KiB | 6.77 MiB/s, done.
Resolving deltas: 100% (307/307), done.
/content/BiT-ImageCaptioning
  Preparing metadata (setup.py) ... done


## Import all libraries

In [2]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import torch
from PIL import Image
from bit_image_captioning.feature_extractors.vinvl import VinVLFeatureExtractor
from bit_image_captioning.pipelines.bert_pipeline import BiTImageCaptioningPipeline
from bit_image_captioning.datasets.ok_vqa_dataset import OKVQADataset
from bit_image_captioning.datasets.ok_vqa_dataloader import OKVQADataLoader

## Configuration

In [ ]:
class BiTConfig:
    """
    Configuration class for BiTImageCaptioningPipeline.
    This class provides all the necessary settings for initializing and running the pipeline.
    """

    # General settings
    checkpoint = "/content/BiT-ImageCaptioning/src/bit_image_captioning/pretrained_model"  # Path to the pretrained model
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # Device for computation (GPU/CPU)

    # Dataset settings
    dataset_path = "ShahadMAlshalawi/ok-vqa-ar-2"  # Path or name of the dataset
    language = "ar"  # Language for questions/answers ("ar" for Arabic, "en" for English)
    split = "validation"  # Dataset split to use ("train", "validation", "test")

    # Image and object detection settings
    add_od_labels = True  # Whether to add object detection labels to input
    max_img_seq_length = 50  # Maximum sequence length for image features

    # Text input settings
    max_seq_length = 70  # Maximum sequence length for text input
    max_seq_a_length = 40  # Maximum sequence length for primary text (e.g., question)
    is_train = False  # Whether the configuration is for training or inference
    mask_prob = 0.15  # Probability of masking tokens during training
    max_masked_tokens = 3  # Maximum number of tokens to mask in a single sequence

    # DataLoader settings
    batch_size = 2  # Number of samples per batch
    num_workers = 1  # Number of workers for data loading
    shuffle = False  # Whether to shuffle the dataset
    pin_memory = False  # Whether to use pinned memory (for CUDA optimization)
    drop_last = False  # Whether to drop the last incomplete batch
    seed = 42  # Random seed for reproducibility

    # Generation settings
    is_decode = True  # Enable decoding (generation mode)
    do_sample = False  # Whether to use sampling for generation
    bos_token_id = None  # Beginning of sentence token ID (will be set by tokenizer)
    pad_token_id = None  # Padding token ID (will be set by tokenizer)
    eos_token_ids = None  # End of sentence token ID(s) (will be set by tokenizer)
    mask_token_id = None  # Masking token ID (will be set by tokenizer)
    max_gen_length = 50  # Maximum length for generated text
    num_beams = 5  # Number of beams for beam search
    temperature = 1.0  # Temperature for sampling (lower values make output more deterministic)
    top_k = 50  # Top-k sampling (0 disables it)
    top_p = 1.0  # Top-p (nucleus) sampling (0 disables it)
    repetition_penalty = 1.0  # Penalty for repeating words (1.0 disables it)
    length_penalty = 1.0  # Penalty for sequence length (used in beam search)
    num_return_sequences = 1  # Number of sequences to return
    num_keep_best = 3  # Number of best sequences to keep

    # Constrained Beam Search (CBS) settings
    use_cbs = False  # Whether to use constrained beam search
    min_constraints_to_satisfy = 0  # Minimum number of constraints to satisfy (if CBS is enabled)

# ........................................................................................
# download Image captioning model
!git lfs install
!git clone https://huggingface.co/jontooy/AraBERT32-Flickr8k $BiTConfig.checkpoint

Git LFS initialized.
Cloning into '/content/BiT-ImageCaptioning/src/bit_image_captioning/pretrained_model'...
remote: Enumerating objects: 19, done.
remote: Total 19 (delta 0), reused 0 (delta 0), pack-reused 19 (from 1)
Unpacking objects: 100% (19/19), 259.13 KiB | 4.47 MiB/s, done.


## OKVQA Dataset

In [5]:
!pip install datasets --quiet

In [6]:
from huggingface_hub import login
from google.colab import userdata
from datasets import load_dataset

In [7]:
# Login to Hugging Face
HF_TOKEN = userdata.get('HF_TOKEN')
login(token=HF_TOKEN)

In [8]:
dataset = load_dataset(BiTConfig.dataset_path,split=BiTConfig.split)
dataset

README.md:   0%|          | 0.00/1.09k [00:00<?, ?B/s]

train-00000-of-00004.parquet:   0%|          | 0.00/373M [00:00<?, ?B/s]

train-00001-of-00004.parquet:   0%|          | 0.00/367M [00:00<?, ?B/s]

train-00002-of-00004.parquet:   0%|          | 0.00/368M [00:00<?, ?B/s]

train-00003-of-00004.parquet:   0%|          | 0.00/378M [00:00<?, ?B/s]

validation-00000-of-00002.parquet:   0%|          | 0.00/419M [00:00<?, ?B/s]

validation-00001-of-00002.parquet:   0%|          | 0.00/414M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/9009 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5046 [00:00<?, ? examples/s]

Dataset({
    features: ['metadata', 'image', 'question', 'answers'],
    num_rows: 5046
})

In [9]:
print(dataset[0])

{'metadata': {'image_id': 297147, 'question_id': 2971475, 'question_type': 'one', 'answer_type': 'other', 'confidence': 3}, 'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=640x480 at 0x7EBAF03CDCF0>, 'question': {'en': 'What sport can you use this for?', 'ar': 'في أي رياضة يمكنك استخدام هذا؟'}, 'answers': {'en': ['race', 'race', 'race', 'race', 'race', 'race', 'motocross', 'motocross', 'ride', 'ride'], 'ar': ['سباق', 'سباق', 'سباق', 'سباق', 'سباق', 'سباق', 'موتوكروس', 'موتوكروس', 'يركب', 'يركب'], 'raw_en': ['racing', 'racing', 'racing', 'racing', 'racing', 'racing', 'motocross', 'motocross', 'riding', 'riding'], 'raw_ar': ['سباق', 'سباق', 'سباق', 'سباق', 'سباق', 'سباق', 'موتوكروس', 'موتوكروس', 'يركب', 'يركب'], 'confidence': ['yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes'], 'id': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]}}


## Extract Features

In [11]:
from bit_image_captioning.feature_extractors.vinvl import VinVLFeatureExtractor
from bit_image_captioning.pipelines.bert_pipeline import BiTImageCaptioningPipeline

feature_extractor = VinVLFeatureExtractor(add_od_labels=BiTConfig.add_od_labels)
pipeline = BiTImageCaptioningPipeline(BiTConfig)

downloading vinvl_vg_x152c4.pth: 100%|█████████▉| 579565/579565.927734375 [00:12<00:00, 45546.11it/s]
downloading VG-SGG-dicts-vgoi6-clipped.json: 100%|█████████▉| 107/107.1904296875 [00:00<00:00, 3631.23it/s]


In [20]:
def extract_features(examples):
  try:
    examples["features"] = feature_extractor(examples["image"])
  except Exception as e:
    examples["features"] = [None]*len(examples)
    print(f"Error while extracting features: {e}")
  return examples


# ..................................................

def features_captions(examples):
  try:
    features,captions = pipeline(examples["image"])
    examples["features"] = features
    examples["captions"] = captions
  except Exception as e:
    examples["captions"] = [None] * len(examples)
    examples["features"]  = [None] * len(examples)
    print(f"Error while extracting features: {e}")
  return examples

# ..................................................

rng = range(0, 2523)
rng = range(2523, 5046)

ds  = dataset.select(rng).map(features_captions,
                              batched=True,
                              batch_size=20,
                              )

checkpoint_dir = f"/content/drive/MyDrive/OKVQA_Features_Checkpoint_{min(rng)}_{max(rng)}"
ds.save_to_disk(checkpoint_dir)

Map:   0%|          | 0/4 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/4 [00:00<?, ? examples/s]

In [21]:
ds[0].keys()

dict_keys(['metadata', 'image', 'question', 'answers', 'features', 'captions'])

In [22]:
ds[0]['features'].keys()

dict_keys(['boxes', 'classes', 'img_feats', 'od_labels', 'scores', 'spatial_features'])

In [23]:
ds[0]['captions'][0].keys()

dict_keys(['caption', 'confidence'])

In [24]:
ds[0]['captions']

[{'caption': 'امراة ترتدي بدلة سوداء وبيضاء تركب دراجتها في الشارع',
  'confidence': 0.33103856444358826},
 {'caption': 'امراة ترتدي بدلة سوداء وبيضاء تركب دراجتها في موقف للسيارات',
  'confidence': 0.3305349051952362},
 {'caption': 'امراة ترتدي بدلة سوداء وبيضاء تركب دراجة ثلاثية العجلات',
  'confidence': 0.32602453231811523}]



---

